In [17]:
import pandas as pd
from xgboost import XGBClassifier

# Load the model with the same parameters as in your training code
best_params = {'learning_rate': 0.2916903245522562, 'n_estimators': 404, 'max_depth': 7, 'min_child_weight': 1, 'subsample': 0.7194530016125948, 'colsample_bytree': 0.8243666347326039, 'gamma': 1.0504744487954727, 'reg_alpha': 0.7543318286365954, 'reg_lambda': 0.035539482725579316, 'scale_pos_weight': 63.0002814746974}
model = XGBClassifier(**best_params, random_state=42, eval_metric='logloss')

# Define the features in the same order as used in training
features = [
    'PlayoffProgScore', 'FTA', 'DREB', 'AST', 'REB%', 'CumulativePlayoffProgScore', 'AvgPIE_NextYearPlayers', 'Performance_NextYearCoach'
]

# Load and prepare the training data
dataset = pd.read_csv('../cleaned_data/dataset.csv')
X_train = dataset[features]
y_train = dataset['PlayoffNextSeason']

# Train the model on all training data
model.fit(X_train, y_train)

# Load year 10 data
year_10_data = pd.read_csv('../cleaned_data/year_10_stats.csv')

# Prepare the prediction data
X_pred = year_10_data[features]

# Get predictions and probabilities
predictions = model.predict(X_pred)
probabilities = model.predict_proba(X_pred)

# Create results DataFrame
results = pd.DataFrame({
    'Team': year_10_data['Team'],
    'Playoff_Prediction': predictions,
    'Playoff_Probability': probabilities[:, 1]  # Probability of positive class (making playoffs)
})

# Sort by probability in descending order
results = results.sort_values('Playoff_Probability', ascending=False)

# Display results
print("\nPlayoff Predictions for Next Season:")
print("=====================================")
for _, row in results.iterrows():
    print(f"{row['Team']}: {row['Playoff_Probability']*100:.2f}% chance of making playoffs")

# Save to CSV
results.to_csv('playoff_predictions_year_10.csv', index=False)
print("\nResults have been saved to 'playoff_predictions_year_10.csv'")


Playoff Predictions for Next Season:
LAS: 99.99% chance of making playoffs
SAS: 99.92% chance of making playoffs
WAS: 99.50% chance of making playoffs
ATL: 99.07% chance of making playoffs
MIN: 98.81% chance of making playoffs
PHO: 98.09% chance of making playoffs
IND: 96.55% chance of making playoffs
CON: 95.88% chance of making playoffs
SEA: 94.56% chance of making playoffs
NYL: 89.61% chance of making playoffs
CHI: 0.53% chance of making playoffs
TUL: 0.27% chance of making playoffs

Results have been saved to 'playoff_predictions_year_10.csv'
